<a href="https://colab.research.google.com/github/BrunoVandekerkhove/problog/blob/master/problog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install problog

In [0]:
from problog.program import PrologString, LogicProgram
from problog.formula import LogicFormula, LogicDAG
from problog.logic import Term
from problog.ddnnf_formula import DDNNF
from problog.cnf_formula import CNF

### Probabilistic Inference using WMC

One of the most performant techniques to compute the marginal or conditional probability of a query given a probabilistic graphical model (PGM) or probabilistic programs is to reduce the problem to weighted model counting (WMC). This entails that a PGM such as a Bayesian network is represented as a propositional knowledge base in conjunctive normal form (CNF) with weights associated to the propositional variables. You will build a small compiler based on such approach.

#### SRL to CNF

ProbLog, a Statistical Relational Learning formalism, is a generalization of PGM that allows one to express complex re- lations. Similar to PGMs, probabilistic inference for ProbLog can be reduced to a Weighted Model Counting task. Read about this approach in Fierens et al. [2015] (sections 1-6.1).

*   ***Task 1.1.1*** Write the encoding for the ProbLog program as logic formula and associated weights.For partial credit youcan restrict the friends relationships to 0.1::friends(a,b) and 0.1::friends(a,c) to avoid cycles (also for the next tasks).
*   ***Task 1.1.2*** Show the intermediate steps to translate the given program to a CNF.

In [6]:
p = PrologString("""
person(a). 
person(b). 
person(c). 
0.2::stress(X) :- person(X). 
0.1::friends(X,Y) :- person(X), person(Y). 
0.3::smokes(X) :- stress(X). 
0.4::smokes(X) :- friends(X,Y), smokes(Y). 
query(smokes(a)).
""")
lf2 = LogicFormula.create_from(p, avoid_name_clash=True, keep_order=True, label_all=True)
# print(LogicFormula.to_prolog(lf2))
dag2 = LogicDAG.create_from(lf2, avoid_name_clash=True, keep_order=True, label_all=True)
# print(dag2)
print(LogicFormula.to_prolog(dag2))
cnf2 = CNF.create_from(dag2)
# for clause in cnf2._clauses:
#     print(clause)
ddnnf2 = DDNNF.create_from(cnf2)
print(ddnnf2.evaluate())

0.2::stress(a).
0.3::smokes(a) :- stress(a).
0.1::friends(a,b).
0.2::stress(b).
0.1::friends(b,c).
0.2::stress(c).
0.3::_problog_smokes_cb_0(c) :- stress(c).
0.1::friends(c,b).
0.4::smokes(a) :- friends(a,b), node_15.
0.1::friends(a,c).
0.4::smokes(a) :- friends(a,c), _problog_smokes_cb_0(c).
0.3::choice(24,0,smokes(b),b).
0.4::choice(33,0,smokes(c),c,b).
0.4::choice(33,0,smokes(b),b,c).
query(smokes(a)).
{smokes(a): 0.06466945075200001}


#### SRL to PGM

A ProbLog program can be translated to an equivalent Bayesian network. In ProbLog, multiple rules with the same head can be considered as a noisy-OR structure in PGM [Díez and Druzdzel, 2006, Sec 4.1.1]. You can also use the ProbLog tutorial for inspiration.


*   ***Task 1.2.1*** Show a complete and correct equivalent Bayesian network for the given ProbLog program and query.

#### PGM to CNF

Read about this approach in Chavira and Darwiche [2008] (sections 1-3, optionally also 4) and familiarize yourself with ENC1 (the encoding of Chavira and Darwiche) and ENC2 (the encoding of Sang, Beam and Kautz).

*   ***Task 1.3.1*** Write the logic encoding for the Bayesian network (with full CPTs) from Task1.2.1 using ENC1.
*   ***Task 1.3.2*** Write the logic encoding for the Bayesian network (with full CPTs) from Task1.2.1 using ENC2.
*   ***Task 1.3.3*** Can you come up with a more compact variant of either ENC1 or ENC2 by exploiting the knowledge that the Bayesian network expresses noisy-OR structures?

#### Weighted Model Counting

WMC can be performed by applying a search algorithm on the CNF or by compiling the CNF into a structure on which WMC can be performed in polynomial time with respect to the size of the structure. An advantage of the standardization to CNF is that multiple model counters can be applied, each with their own advantages and disadvantages. An exhaustive overview of exact model counters is available on http://beyondnp.org.

*   ***Task 1.4.1*** Use the SDD package 2 and one other exact weighted model counter, and apply them to the four logic encodings from the previous tasks (for the given query). Compute and report the WMC (at least 4 digits behind the decimal point). Can you interpret the WMC as probabilities?
*   ***Task 1.4.2*** What is the smallest circuit for each model you found and using which hyperparameters?
*   ***Task 1.4.3*** Use WMC to compute the probabilities P(smokes(a)) and P(smokes(a)|friends(a, b) = ⊤, friends(a, c) = ⊤). Explain how you perform this computation and show the results.
*   ***Task 1.4.4*** Explain in one paragraph the main theoretical differences between the two weighted model counters.

### Lifted Inference

In this part, you are tasked with applying lifted inference concepts to the models discussed before. Use the rules intro- duced in the lecture slides. You can also use the approach by Van den Broeck et al. [2014].

*   ***Task 2.1*** Write down the formula to compute the probability for query P(smokes(a)) using the lifted inference rules for probabilistic databases as seen in the lecture.
*   ***Task 2.2*** Can you apply lifted inference techniques from probabilistic databases to construct alternative encodings for the ProbLog program (that can be expressed as a CNF). Show the resulting circuit size and compare to the previous circuit sizes.

### Parameter Learning

One of the key tasks in machine learning is learning the parameters that fit a given dataset. In this task you implement your own Expectation-Maximisation learning algorithm based on Gutmann et al. [2011].

*   ***Task 3.1*** Included with the assignment is the file data.pl with 1000 examples. Use these examples and your encoding together with the above formula to estimate parameters pn to learn who is friends with who. Explain your algorithm and show the learned parameters.
*   ***Task 3.2*** Repeat the previous task with the first 100 examples. Show the results and explain the difference with the previous task.